In [1]:
import sys
sys.path.append('../src')

In [2]:
import torch
from transformer_wrappers.wrappers import CausalLMWrapper
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
COLOURS = [f'C{i}' for i in range(10)]
STYLES = ['solid', 'dotted', 'dashed', 'dashdot']

In [5]:
import os
TOKEN = os.environ['HUGGINGFACE_TOKEN']

In [6]:
# MODEL = 'gpt2-xl'
# MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'  
# MODEL = 'meta-llama/Llama-2-7b-hf'
#MODEL = 'google/gemma-7b'
MODEL = 'EleutherAI/pythia-14m'
MODEL_CONFIGS = {
    'torch_dtype': torch.bfloat16,
    'attn_implementation': 'eager',
    'device_map': DEVICE,
    'token': TOKEN,
}

# TOKENIZER = 'gpt2-xl'
# TOKENIZER = 'mistralai/Mistral-7B-Instruct-v0.2'  
# TOKENIZER = 'meta-llama/Llama-2-7b-hf'
#TOKENIZER = 'google/gemma-7b'
TOKENIZER = 'EleutherAI/pythia-14m'
TOKENIZER_CONFIGS = {'token': TOKEN}

In [7]:
model = CausalLMWrapper.from_pretrained(MODEL, 
                                        model_kwargs=MODEL_CONFIGS, 
                                        tokenizer_name_or_path=TOKENIZER, 
                                        tokenizer_kwargs=TOKENIZER_CONFIGS,
                                        )
model.enable_benchmarking()

In [8]:
input_string = 'The capital of Italy is'

In [9]:
input_encodings = model.tokenizer(input_string, return_tensors='pt').to(DEVICE)

In [10]:
output_wrapper = model(
    **input_encodings,
    return_dict=True,
    output_attentions=True,
    use_cache=True,
    output_hidden_stsates=True,
    return_attention_output=True,  # Self-attention layer output
    return_feed_forward_output=True
)

In [11]:
output = model.generate(input_encodings.input_ids,
                        do_sample=False, 
                        max_length=25, 
                        return_dict_in_generate=True, 
                        output_attentions=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
output = model.generate(input_encodings.input_ids,
                        do_sample=False, 
                        max_length=25, 
                        return_dict_in_generate=True, 
                        output_attentions=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [13]:
model.tokenizer.decode(output['sequences'].squeeze())

'The capital of Italy is,,<|endoftext|>'